use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import random
import numpy as np
import pandas as pd
# import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 1024
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id].to('cuda:2')
            product_emb = product_embeddings[batch_product_id].to('cuda:2')
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge valid score

In [ ]:
FIELD_NAME = 'seqmlp_scores'

In [6]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [7]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [8]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_DE/product_embeddings_2023-06-01-21-48-50.pt'
DE_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_DE/valid_embeddings_2023-06-01-21-48-58.pt'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_JP/product_embeddings_2023-06-01-21-49-56.pt'
JP_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_JP/valid_embeddings_2023-06-01-21-50-04.pt'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_UK/product_embeddings_2023-06-01-21-51-01.pt'
UK_valid_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates_phase2/query_embeddings/SeqMLP/kdd_cup_2023_UK/valid_embeddings_2023-06-01-21-51-09.pt'

In [9]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/b5aeac4e5b9ff0518bbcb59a28086594'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/2536617955df215e0047f5b220d1c012'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/8d133ea55ad67bd3efd625dfeff0fb1d'

In [10]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [11]:
merged_candidates = read_merged_candidates_feature()
valid_sessions = read_valid_sessions()
EMBED_DIM = 256
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [14]:
# sess embeddings 
valid_DE_query_emb = torch.load(DE_valid_embeddings_path, map_location='cpu')
valid_JP_query_emb = torch.load(JP_valid_embeddings_path, map_location='cpu')
valid_UK_query_emb = torch.load(UK_valid_embeddings_path, map_location='cpu')
valid_query_embeddings = torch.empty(len(valid_sessions), EMBED_DIM)
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'DE'].index).tolist()] = valid_DE_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'JP'].index).tolist()] = valid_JP_query_emb
valid_query_embeddings[(valid_sessions[valid_sessions['locale'] == 'UK'].index).tolist()] = valid_UK_query_emb

In [15]:
# product_embeddings 
DE_product_emb = torch.load(DE_product_embeddings_path, map_location='cpu')
JP_product_emb = torch.load(JP_product_embeddings_path, map_location='cpu')
UK_product_emb = torch.load(UK_product_embeddings_path, map_location='cpu')
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [16]:
DE_product_emb.shape

torch.Size([518328, 256])

In [17]:
merged_candidates_seqmlp = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [18]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [19]:
# merged_candidates_seqmlp_g = cudf.from_pandas(merged_candidates_seqmlp)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [20]:
# merged_candidates_seqmlp_score_g = merged_candidates_seqmlp_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_seqmlp_score_g['dataset_id'] = merged_candidates_seqmlp_score_g['dataset_id'].fillna(0)
# merged_candidates_seqmlp_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_seqmlp_score_g = merged_candidates_seqmlp_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_seqmlp_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_seqmlp_score = merged_candidates_seqmlp_score_g.to_pandas()
# assert len(merged_candidates_seqmlp_score) == len(merged_candidates)

In [ ]:
merged_candidates_seqmlp_score = merged_candidates_seqmlp.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_seqmlp_score['dataset_id'] = merged_candidates_seqmlp_score['dataset_id'].fillna(0)
merged_candidates_seqmlp_score.drop(columns=['locale'], inplace=True)
merged_candidates_seqmlp_score = merged_candidates_seqmlp_score.sort_values(by=['sess_id', 'product'])
merged_candidates_seqmlp_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_seqmlp_score) == len(merged_candidates)

In [21]:
# del merged_candidates_seqmlp_g
# del product_id_df_g
# del merged_candidates_seqmlp_score_g

In [22]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] = \
        merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_272043/1863069582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_seqmlp_score['dataset_id'][merged_candidates_seqmlp_score['sess_locale'] == locale] = \


In [24]:
merged_candidates_seqmlp_score[FIELD_NAME] = get_scores(merged_candidates_seqmlp_score, valid_query_embeddings, product_embeddings)

100%|██████████| 41215/41215 [05:15<00:00, 130.65it/s]


In [25]:
normalize_scores(merged_candidates_seqmlp_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [26]:
merged_candidates[FIELD_NAME] = merged_candidates_seqmlp_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_seqmlp_score['normalized_'+FIELD_NAME]

In [34]:
cast_dtype(merged_candidates)
merged_candidates.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [32]:
# verify sasrec scores
merged_candidates[merged_candidates['sess_id'] == 3013].sort_values(by=['sasrec_scores_2'], ascending=False)[['sess_id', 'sess_locale', 'product', FIELD_NAME, 'normalized_'+FIELD_NAME, 'sasrec_scores_2', 'normalized_sasrec_scores_2']].iloc[:20]

,sess_id,sess_locale,product,seqmlp_scores,seqmlp_normalized_scores,sasrec_scores_3,sasrec_normalized_scores_3
706060,3013,JP,B06X6BJ13P,13.300402,0.08856,12.641310,0.183014
706058,3013,JP,B06VSKB9DB,13.193668,0.079594,12.610657,0.177489
706059,3013,JP,B06WV7B1DR,12.940981,0.061822,12.428663,0.147956
706057,3013,JP,B06VSCGTYJ,12.887313,0.058591,12.089650,0.105415
706098,3013,JP,B07S8VWT5B,14.072466,0.191663,11.718424,0.072724
706150,3013,JP,B0956X785M,12.787196,0.053009,11.063332,0.037773
706090,3013,JP,B07JB6Q58L,11.666172,0.017278,10.303680,0.017671
706144,3013,JP,B0927ZDJTL,11.755957,0.018901,10.241162,0.016600
706197,3013,JP,B09Y8G6FJJ,12.823019,0.054943,10.113563,0.014612
706039,3013,JP,B0161YPB8Y,11.818468,0.020121,10.005676,0.013117
